# Data Loader

It'll make things look a bit cleaner to load the data in a separate script from the report generator.


In [1]:
# Step 0: Import all the things!

import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime as dt
import geopy
from geopy.extra.rate_limiter import RateLimiter
from IPython.display import HTML, Markdown, clear_output
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import urllib3

%run controls.ipynb

def datestamp(date):
    """Convert a datetime object into a date stamp: MM/DD/YYYY"""
    return f"{date.month}/{date.day}/{date.year}"


# Step 1: Download a copy of the web page that contains the data I want.

The state has moved to an arcgis data source.  Looks a lot nicer than an html table, but it's also harder to scrape.  I'll try to figure out how to connect to that data later, but I was able to copy and paste this text from here: https://txdshs.maps.arcgis.com/apps/opsdashboard/index.html#/ed483ecd702b4298ab01e8b9cafc8b83

In [2]:
# TODO: Automate this with Selenium.

text_copy = """

"""

# Step 2: Clean up the text I copied and convert it into a proper DataFrame object.

In [3]:
today = dt.datetime.today()
today_text = datestamp(today)

num_cases = []
text_list = text_copy.strip().split('\n')
for n in range(0, len(text_list), 4):
    num_cases.append({
        "county": text_list[n].replace(" County", ""),
        "date": today_text,
        "num_cases": text_list[n + 2].replace(",", "")
    })

df_num_cases = pd.DataFrame(num_cases)
df_num_cases

,county,date,num_cases
0,Harris,4/11/2020,3261
1,Dallas,4/11/2020,1537
2,Tarrant,4/11/2020,705
3,Travis,4/11/2020,690
4,Bexar,4/11/2020,665
...,...,...,...
172,Mitchell,4/11/2020,1
173,Pecos,4/11/2020,1
174,Rains,4/11/2020,1
175,Stephens,4/11/2020,1


# Step 3: Add the latitude and longitude to each row.

In [4]:
locator = geopy.Nominatim(user_agent="myGeocoder")
geocode = RateLimiter(locator.geocode, min_delay_seconds=0.5)

df_counties = pd.read_csv("county_coords.csv")

def get_coordinates(county_name):
    global df_counties
    exists = len(df_counties.loc[df_counties["county"] == county_name]) != 0
    if exists:
        return [{"latitude":r[0], "longitude":r[1]} for r in df_counties.loc[df_counties["county"] == county_name][["latitude", "longitude"]].values][0]
    else:
        point = geocode(county_name + " County, TX")
        df_county = pd.DataFrame([[county_name, point.latitude, point.longitude]], columns=["county", "latitude", "longitude"])
        df_counties = df_counties.append(df_county)
        return {"latitude":point.latitude, "longitude":point.longitude}

df_num_cases["point"] = (df_num_cases["county"]).apply(get_coordinates) #geocode)
##df_num_cases["point"] = (df_num_cases["county"]).apply(geocode)

# I am substituting (28.082612, -94.936773) for the *official* coordinates of the Gulf of Mexico to better position it on the map.
##df_num_cases[['latitude', 'longitude']] = pd.DataFrame([
##    (p.latitude, p.longitude) if p != None else (28.082612, -94.936773) # list(locator.geocode("Gulf of Mexico"))[1]
##    for p in df_num_cases["point"].tolist()])
df_num_cases[['latitude', 'longitude']] = pd.DataFrame([
    (p["latitude"], p["longitude"]) if p != None else (28.082612, -94.936773) # list(locator.geocode("Gulf of Mexico"))[1]
    for p in df_num_cases["point"].tolist()])
df_num_cases = df_num_cases.drop(["point"], axis=1)

# Save the county coordinates to preserve any new data.
df_counties.set_index("county", inplace=True)
df_counties.to_csv("county_coords.csv")

# Step 4: Concatenate today's data with the full dataset.

In [5]:
# TODO: Running this more than once will add duplicates to the data.  I can make this nicer by performing some kind of distinct() function on the data.

df_num_cases = pd.concat([pd.read_csv("data.csv"), df_num_cases])

# Step 5: Save the new dataset.

In [6]:
df_num_cases.columns = ["unnamed", "county", "date", "num_cases", "latitude", "longitude"]
df_num_cases = df_num_cases.drop(columns=["unnamed"], axis=1)

# TODO: Always make a backup before running this.
df_num_cases.to_csv("data.csv")

# Now we are ready to play with it.

In [8]:
# This chunk of code will fix the date column to make it text-sortable, by zero-padding the pieces of the date.
# If this virus continues into 2021 I will need to amend the date column to be YYYYMMDD instead of MM/DD/YYYY.
return
import pandas as pd

def fix_date(date_text):
    pieces = date_text.split("/")
    pieces[0] = pieces[0].zfill(2)
    pieces[1] = pieces[1].zfill(2)
    pieces[2] = pieces[2].zfill(2)
    return pieces[0] + "/" + pieces[1] + "/" + pieces[2]

df_data = pd.read_csv("data.csv")
df_data.columns = ["unnamed", "county", "date", "num_cases", "latitude", "longitude"]
df_data = df_data.drop(columns=["unnamed"], axis=1)

df_data["date"] = df_data["date"].apply(fix_date)

df_data.to_csv("data.csv")